In [2]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

In [4]:
!pip install langchain-openai

In [3]:
# If you are using multiple LLMs and want to debug, test, and monitor
# AI app performance, we can use LagGmith.

import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")


··········


In [5]:
# Import chat model from langchain

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")


In [6]:
# Import embeddings class and intialize an embeddings object
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [7]:
# Import the vector store and initialize it with the embeddings model

from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)


## Building a simple indexing pipeline and RAG chain to create an app that answers questions about the website's content, following the LANGCHAIN documentation


In [ ]:
# Import libraries
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain.schema import Document

# Load the web content and split it into chunks

# Load from a specific web path
loader = WebBaseLoader(
    web_paths=("https://python.langchain.com/docs/tutorials/rag/",),
          bs_kwargs={"parse_only": bs4.SoupStrainer(class_="main-content")},
  )

documents = loader.load()

# Split the text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

# Define a few sentences as the web content for a quick check
simple_documents = [
    Document(page_content="Overview"),
    Document(page_content="A typical RAG application has two main components:"),

]


# Index chunks of text in the vector store
_ = vector_store.add_documents(simple_documents)

In [ ]:
# Define question prompt
prompt = hub.pull("rlm/rag-prompt")

# Define application state
class State(TypedDict):
  """
  Represents the application state as a dictionary with a specific keys:

  keys:
      question (str): the input question as a string.
      context (List[Document]): a list of documents representing the relevant context.
      answer (str): the generated or retrieved answer as a string.

  """
  question: str
  context: List[Document]
  answer: str


In [ ]:
# Define application steps

# 1- Define retrieve function
def retrieve(state: State):
  """
  Performs retrieval of relevant documents from the vector database based on the user's question.

  Parameters:
      state: A dictionary containing at least the 'question' key with the user's query as the value.

  Returns:
      context: A list of documents representing the relevant context.
  """
  retrieved_documents = vector_store.similarity_search(state["question"])
  return {"context": retrieved_documents}


# 2- Define generate function

def generate(state: State):
  """
  Performs generation based on the user's question and the relevant context.

  Parameters:
      state: A dictionary containing:
          - 'question': The user's query as a string.
          - 'context': The relevant context (e.g., a list of retrieved documents).

  Returns:
      answer: The generated or retrieved answer as a string.
  """
  documnets_content = "\n\n".join([doc.page_content for doc in state["context"]])
  prompt_with_context = prompt.invoke({"question": state["question"], "context": documnets_content})
  response = llm.invoke(prompt_with_context)
  return {"answer": response.content}

In [ ]:
# Run application
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [ ]:
response = graph.invoke({"question": "What is Task Decomposition?"})
print(response["answer"])